import cv2
import numpy as np

img = cv2.imread('box.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)

dst = cv2.cornerHarris(gray, 4, 5, 0.04)
dst = cv2.dilate(dst, None)

img[dst > 0.01*dst.max()]=[0, 0, 0]

cv2.imshow('Harris Corners', img)
cv2.waitKey(1)
cv2.destroyAllWindows()

In [1]:
# 모서리 인식하기 1

import cv2
import numpy as np

img = cv2.imread('box.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)

dst = cv2.cornerHarris(gray, 4, 5, 0.04) 
dst = cv2.dilate(dst, None)

img[dst > 0.01*dst.max()]=[0, 0, 0]

cv2.imshow('Harris Corners', img) 
cv2.waitKey(1) 

-1

In [9]:
# 모서리 인식하기 2

import cv2
import numpy as np

img = cv2.imread('box.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)

corners = cv2.goodFeaturesToTrack(gray, 7, 0.05, 25) 
corners = np.float32(corners)

for item in corners:
    x, y = item[0]
    cv2.circle(img, (x,y), 5, 255, -1)

cv2.imshow('TOP', img) 
cv2.waitKey(1) 

-1

In [11]:
# 모서리 인식하기 3  orb 사용 (실패..ㅎ  --> 버전이 맞지 않는 것 같음..)
import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('box.jpeg',0)

# Initiate STAR detector
orb = cv2.ORB()

# find the keypoints with ORB
kp = orb.detect(img,None)

# compute the descriptors with ORB
kp, des = orb.compute(img, kp)

# draw only keypoints location,not size and orientation
img2 = cv2.drawKeypoints(img,kp,color=(0,255,0), flags=0)
plt.imshow(img2),plt.show()

TypeError: Incorrect type of self (must be 'Feature2D' or its derivative)

In [12]:
# sift 도 먹히지 않음.. 특허를 받은거라 뭐..쓸 수 없다고 뜸ㅎㅎ

import numpy as np
import cv2
from matplotlib import pyplot as plt

img1 = cv2.imread('box.jpeg',0)          # queryImage
img2 = cv2.imread('box_in_scene.jpeg',0) # trainImage

# Initiate SIFT detector
sift = cv2.SIFT()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# BFMatcher with default params
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])

# cv2.drawMatchesKnn expects list of lists as matches.
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,flags=2)

plt.imshow(img3),plt.show()

AttributeError: module 'cv2' has no attribute 'SIFT'

In [16]:
#얼굴 모자이크 인식 --> 모자이크 안됨..

import cv2

def nothing(x):
    pass
cap = cv2.VideoCapture('facevideo.mp4')
cv2.namedWindow('Face')

face_cascade = cv2.CascadeClassifier()
face_cascade.load('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_default.xml')

cv2.createTrackbar('BoxW', 'Face', 30, 255, nothing)
cv2.createTrackbar('BoxH', 'Face', 30, 255, nothing)

while(True):
    
    boxW = cv2.getTrackbarPos('BoxW', 'Face')
    boxH = cv2.getTrackbarPos('BoxH', 'Face')
    
    ret, frame = cap.read()

    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    grayframe = cv2.equalizeHist(grayframe) # 히그토그램 균일화가 적용(전반적으로 밝아짐)

    faces = face_cascade.detectMultiScale(grayframe, 1.1, 3, 0, (30, 30))

    for (x,y,w,h) in faces:
        cv2.rectangle(grayframe, (x,y), (x+w,y+h), (0,255,0), 3, 4, 0)

    cv2.imshow('Face',grayframe)
    
    if cv2.waitKey(1) == 27:
        break;
        
cap.release()
cv2.destroyWindow('Face')

In [5]:
# 이미지에서 눈동자인식 --> 안 됨.
import math
import cv2
import numpy as np

img = cv2.imread('face.jpg')
scaling_factor = 0.7

img = cv2.resize(img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
cv2.imshow('Face', img)
gray = cv2.cvtColor(~img, cv2.COLOR_BGR2GRAY)

ret, thresh_gray = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY)
# 윤곽선을 찾는 것
im2, contours, hierarchy = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    area = cv2.contourArea(contour)
    rect = cv2.boundingRect(contour)
    x, y, width, height = rect
    radius = 0.25 * (width + height)
    
    area_condition = (100 <= area <= 200)
    symmetry_condition = (abs(1 - float(width) / float(height)) <= 0.2)
    fill_condition = (abs(1 - (area / (math.pi * math.pow(radius, 2.0)))) <= 0.3)
    
    if area_condition and symmetry_condition and fill_condition:
        cv2.circle(img,(int(x + radius), int(y + radius)), int(1.3 * radius), (0,180,0), -1)
        
cv2.imshow('Pupil Detector', img)

c=cv2.waitKey()
cv2.destroyAllWindows()

In [9]:
# 눈 인식하는 영상

import cv2 
import numpy as np 
 
face_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_alt.xml') 
eye_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml') 
if face_cascade.empty(): 
  raise IOError('Unable to load the face cascade classifier xml file') 
 
if eye_cascade.empty(): 
  raise IOError('Unable to load the eye cascade classifier xml file') 
 
cap = cv2.VideoCapture('facevideo.mp4')
cv2.namedWindow('Face')
ds_factor = 0.5 
 
while True: 
    ret, frame = cap.read() 
    frame = cv2.resize(frame, None, fx=ds_factor, fy=ds_factor, interpolation=cv2.INTER_AREA) 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
 
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=1) 
    for (x,y,w,h) in faces: 
        roi_gray = gray[y:y+h, x:x+w] 
        roi_color = frame[y:y+h, x:x+w] 
        eyes = eye_cascade.detectMultiScale(roi_gray) 
        for (x_eye,y_eye,w_eye,h_eye) in eyes: 
            center = (int(x_eye + 0.5*w_eye), int(y_eye + 0.5*h_eye)) 
            radius = int(0.3 * (w_eye + h_eye)) 
            color = (0, 255, 0) 
            thickness = 3 
            cv2.circle(roi_color, center, radius, color, thickness) 
 
    cv2.imshow('Eye Detector', frame) 
 
    c = cv2.waitKey(1) 
    if c == 27: 
        break 
 
cap.release() 
cv2.destroyAllWindows() 

In [2]:
# 가면씌우는 코드

import cv2 
import numpy as np 
 
face_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_alt.xml') 
eye_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml') 
 
if face_cascade.empty(): 
  raise IOError('Unable to load the face cascade classifier xml file') 
if eye_cascade.empty(): 
  raise IOError('Unable to load the eye cascade classifier xml file') 
 
cap = cv2.VideoCapture('facevideo.mp4')
sunglasses_img = cv2.imread('mask.jpg')

while True:
    ret, frame = cap.read() 
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    vh, vw = frame.shape[:2]
    vh, vw = int(vh), int(vw)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=1)
    centers = []

    for (x,y,w,h) in faces: 
        roi_gray = gray[y:y+h, x:x+w] 
        roi_color = frame[y:y+h, x:x+w] 
        eyes = eye_cascade.detectMultiScale(roi_gray) 
        for (x_eye,y_eye,w_eye,h_eye) in eyes: 
            centers.append((x + int(x_eye + 0.5*w_eye), y + int(y_eye + 0.5*h_eye))) 
    
    if len(centers) > 1: # if detects both eyes
        h, w = sunglasses_img.shape[:2]
        # Extract the region of interest from the image 
        eye_distance = abs(centers[1][0] - centers[0][0])
        # Overlay sunglasses; the factor 2.12 is customizable depending on the size of the face 
        sunglasses_width = 2.12 * eye_distance
        scaling_factor = sunglasses_width / w
        print(scaling_factor, eye_distance)
        overlay_sunglasses = cv2.resize(sunglasses_img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)

        x = centers[0][0] if centers[0][0] < centers[1][0] else centers[1][0] 
     
        # customizable X and Y locations; depends on the size of the face 
        x -= int(0.26*overlay_sunglasses.shape[1])
        y += int(0.26*overlay_sunglasses.shape[0])
        
        h, w = overlay_sunglasses.shape[:2]
        h, w = int(h), int(w)
        frame_roi = frame[y:y+h, x:x+w]
        # Convert color image to grayscale and threshold it 
        gray_overlay_sunglassess = cv2.cvtColor(overlay_sunglasses, cv2.COLOR_BGR2GRAY) 
        ret, mask = cv2.threshold(gray_overlay_sunglassess, 180, 255, cv2.THRESH_BINARY_INV) 

        # Create an inverse mask 
        mask_inv = cv2.bitwise_not(mask) 

        try:
            # Use the mask to extract the face mask region of interest 
            masked_face = cv2.bitwise_and(overlay_sunglasses, overlay_sunglasses, mask=mask) 
            # Use the inverse mask to get the remaining part of the image 
            masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv) 
        except cv2.error as e:
            print('Ignoring arithmentic exceptions: '+ str(e))
            #raise e

        # add the two images to get the final output 
        frame[y:y+h, x:x+w] = cv2.add(masked_face, masked_frame)
    else:
        print('Eyes not detected')

    
    cv2.imshow('Eye Detector', frame) 
    c = cv2.waitKey(1) 
    if c == 27: 
        break 

cap.release() 
cv2.destroyAllWindows()

Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
Eyes not detected
0.4380165289256198 100
Ignoring arithmentic exceptions: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\core\src\arithm.cpp:241: error: (-215) (mtype == 0 || mtype == 1) && _mask.sameSize(*psrc1) in function cv::binary_op



error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\core\src\arithm.cpp:659: error: (-209) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function cv::arithm_op


In [19]:
import math 
import cv2 
import numpy as np

eye_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml') 
if eye_cascade.empty():
    raise IOError('Unable to load the eye cascade classifier xml file')

cap = cv2.VideoCapture('facevideo.mp4')
ds_factor = 0.5
ret, frame = cap.read()
contours = []

while True: 
    ret, frame = cap.read() 
    frame = cv2.resize(frame, None, fx=ds_factor, fy=ds_factor, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=1)
    for (x_eye, y_eye, w_eye, h_eye) in eyes:
        pupil_frame = gray[y_eye:y_eye + h_eye, x_eye:x_eye + w_eye]
        ret, thresh = cv2.threshold(pupil_frame, 80, 255, cv2.THRESH_BINARY)
        cv2.imshow("threshold", thresh)
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        print(contours)

    for contour in contours:
        area = cv2.contourArea(contour)
        rect = cv2.boundingRect(contour)
        x, y, w, h = rect
        radius = 0.15 * (w + h)

        area_condition = (100 <= area <= 200)
        symmetry_condition = (abs(1 - float(w) / float(h)) <= 0.2)
        fill_condition = (abs(1 - (area / (math.pi * math.pow(radius, 2.0)))) <= 0.4)
        cv2.circle(frame, (int(x_eye + x + radius), int(y_eye + y + radius)), int(1.3 * radius), (0, 180, 0), -1)

    cv2.imshow('Pupil Detector', frame)
    c = cv2.waitKey(1) 
    if c == 27: 
        break
  
cap.release() 
cv2.destroyAllWindows()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[array([[[92,  7]],

       [[92,  9]],

       [[91, 10]],

       [[84, 10]],

       [[84, 17]],

       [[88, 17]],

       [[89, 18]],

       [[92, 18]],

       [[94, 20]],

       [[96, 20]],

       [[96,  7]]], dtype=int32)]
[]
[]
[]
[]
[]
[array([[[47, 52]],

       [[47, 54]],

       [[54, 54]],

       [[54, 52]]], dtype=int32), array([[[36, 52]],

       [[34, 54]],

       [[31, 54]],

       [[38, 54]],

       [[38, 52]]], dtype=int32)]
[]
[]
[array([[[ 7, 48]],

       [[ 7, 51]],

       [[10, 51]],

       [[10, 48]]], dtype=int32), array([[[51, 45]],

       [[49, 47]],

       [[47, 47]],

       [[46, 48]],

       [[43, 48]],

       [[42, 49]],

       [[35, 49]],

       [[34, 50]],

       [[31, 50]],

       [[30, 51]],

       [[23, 51]],

       [[21, 49]],

       [[17, 49]],

       [[17, 50]],

       [[16, 51]],

       [[16, 52]],

       [[52, 52]],

       [[52, 45]]], dtype=int32), array([[[28, 12]],

    

[array([[[ 5, 56]],

       [[ 4, 57]],

       [[ 4, 60]],

       [[ 3, 61]],

       [[ 3, 62]],

       [[ 4, 63]],

       [[ 4, 64]],

       [[53, 64]],

       [[53, 63]],

       [[55, 61]],

       [[55, 59]],

       [[56, 58]],

       [[53, 58]],

       [[52, 57]],

       [[46, 57]],

       [[45, 56]],

       [[43, 56]],

       [[42, 57]],

       [[37, 57]],

       [[36, 58]],

       [[32, 58]],

       [[31, 59]],

       [[27, 59]],

       [[26, 58]],

       [[25, 58]],

       [[24, 59]],

       [[23, 58]],

       [[16, 58]],

       [[15, 57]],

       [[ 7, 57]],

       [[ 6, 56]]], dtype=int32), array([[[26, 23]],

       [[26, 24]],

       [[27, 25]],

       [[27, 26]],

       [[31, 26]],

       [[32, 25]],

       [[31, 24]],

       [[27, 24]]], dtype=int32)]
[array([[[17, 89]],

       [[16, 90]],

       [[16, 93]],

       [[15, 94]],

       [[15, 95]],

       [[16, 96]],

       [[65, 96]],

       [[67, 94]],

       [[67, 92]],

       [[6

[array([[[46, 16]],

       [[47, 15]],

       [[48, 15]],

       [[54, 21]],

       [[54, 22]],

       [[55, 23]],

       [[55, 26]],

       [[54, 27]],

       [[53, 27]],

       [[51, 25]],

       [[51, 23]],

       [[49, 21]],

       [[48, 21]],

       [[46, 19]]], dtype=int32), array([[[30,  5]],

       [[29,  6]],

       [[28,  6]],

       [[28,  7]],

       [[27,  8]],

       [[26,  8]],

       [[26,  9]],

       [[25, 10]],

       [[25, 14]],

       [[24, 15]],

       [[24, 16]],

       [[23, 17]],

       [[23, 18]],

       [[22, 19]],

       [[22, 20]],

       [[21, 21]],

       [[20, 21]],

       [[24, 21]],

       [[25, 22]],

       [[28, 22]],

       [[29, 23]],

       [[31, 23]],

       [[32, 24]],

       [[35, 24]],

       [[36, 25]],

       [[38, 25]],

       [[39, 26]],

       [[44, 26]],

       [[45, 27]],

       [[47, 27]],

       [[48, 28]],

       [[49, 27]],

       [[50, 27]],

       [[52, 29]],

       [[53, 29]],

     

[array([[[47, 38]],

       [[46, 39]],

       [[42, 39]],

       [[41, 40]],

       [[40, 40]],

       [[39, 41]],

       [[38, 41]],

       [[37, 42]],

       [[35, 42]],

       [[35, 43]],

       [[31, 47]],

       [[31, 49]],

       [[30, 50]],

       [[29, 50]],

       [[31, 52]],

       [[32, 52]],

       [[33, 51]],

       [[37, 51]],

       [[38, 50]],

       [[42, 50]],

       [[43, 49]],

       [[50, 49]],

       [[51, 48]],

       [[55, 48]],

       [[56, 49]],

       [[63, 49]],

       [[63, 48]],

       [[62, 47]],

       [[62, 46]],

       [[61, 46]],

       [[60, 45]],

       [[59, 45]],

       [[58, 44]],

       [[57, 44]],

       [[54, 41]],

       [[53, 41]],

       [[52, 40]],

       [[51, 40]],

       [[50, 39]],

       [[49, 39]],

       [[48, 38]]], dtype=int32), array([[[38, 19]],

       [[39, 18]],

       [[53, 18]],

       [[54, 19]],

       [[57, 19]],

       [[58, 20]],

       [[60, 20]],

       [[61, 21]],

     

[array([[[42, 37]],

       [[43, 36]],

       [[44, 37]],

       [[43, 38]]], dtype=int32), array([[[ 0, 20]],

       [[ 0, 56]],

       [[56, 56]],

       [[56, 31]],

       [[53, 31]],

       [[52, 30]],

       [[52, 29]],

       [[46, 29]],

       [[45, 30]],

       [[42, 30]],

       [[41, 31]],

       [[40, 31]],

       [[39, 32]],

       [[31, 32]],

       [[30, 33]],

       [[32, 35]],

       [[33, 35]],

       [[34, 36]],

       [[37, 36]],

       [[38, 35]],

       [[39, 36]],

       [[40, 36]],

       [[42, 38]],

       [[41, 39]],

       [[40, 39]],

       [[39, 40]],

       [[37, 40]],

       [[36, 41]],

       [[32, 41]],

       [[31, 40]],

       [[28, 40]],

       [[27, 39]],

       [[25, 39]],

       [[24, 38]],

       [[24, 37]],

       [[23, 37]],

       [[22, 36]],

       [[21, 37]],

       [[15, 37]],

       [[14, 36]],

       [[13, 36]],

       [[12, 35]],

       [[10, 35]],

       [[ 8, 33]],

       [[ 8, 32]],

     

...

[array([[[ 0,  0]],

       [[ 0, 19]],

       [[19, 19]],

       [[19, 14]],

       [[16, 14]],

       [[15, 13]],

       [[ 9, 13]],

       [[ 8, 12]],

       [[ 8, 11]],

       [[ 7, 10]],

       [[ 6, 10]],

       [[ 3,  7]],

       [[ 5,  5]],

       [[ 6,  5]],

       [[ 7,  4]],

       [[ 8,  4]],

       [[ 9,  3]],

       [[10,  3]],

       [[11,  2]],

       [[13,  2]],

       [[16,  5]],

       [[17,  5]],

       [[19,  7]],

       [[19,  0]]], dtype=int32)]
[array([[[11, 24]],

       [[12, 23]],

       [[13, 24]],

       [[12, 25]]], dtype=int32), array([[[21, 20]],

       [[22, 19]],

       [[28, 19]],

       [[29, 20]],

       [[31, 20]],

       [[32, 21]],

       [[34, 21]],

       [[35, 22]],

       [[38, 22]],

       [[39, 23]],

       [[41, 23]],

       [[43, 25]],

       [[44, 25]],

       [[45, 26]],

       [[46, 26]],

       [[48, 28]],

       [[49, 28]],

       [[52, 31]],

       [[52, 32]],

       [[51, 33]],

       [[5

[array([[[ 6, 28]],

       [[ 7, 27]],

       [[ 9, 27]],

       [[10, 28]],

       [[ 8, 30]],

       [[ 7, 30]],

       [[ 6, 29]]], dtype=int32), array([[[22, 20]],

       [[23, 19]],

       [[25, 19]],

       [[26, 20]],

       [[31, 20]],

       [[32, 21]],

       [[33, 21]],

       [[34, 22]],

       [[35, 22]],

       [[36, 23]],

       [[38, 23]],

       [[39, 24]],

       [[41, 24]],

       [[42, 25]],

       [[43, 25]],

       [[44, 26]],

       [[45, 26]],

       [[46, 27]],

       [[47, 27]],

       [[48, 28]],

       [[49, 28]],

       [[51, 30]],

       [[51, 31]],

       [[52, 31]],

       [[53, 32]],

       [[50, 35]],

       [[48, 35]],

       [[47, 36]],

       [[46, 36]],

       [[44, 34]],

       [[44, 33]],

       [[42, 33]],

       [[40, 31]],

       [[39, 31]],

       [[38, 30]],

       [[38, 29]],

       [[37, 30]],

       [[36, 30]],

       [[32, 34]],

       [[26, 34]],

       [[25, 33]],

       [[24, 33]],

     

[array([[[11, 22]],

       [[13, 22]],

       [[14, 23]],

       [[14, 24]],

       [[13, 25]],

       [[12, 25]],

       [[11, 26]],

       [[ 9, 26]],

       [[ 8, 25]]], dtype=int32), array([[[47, 14]],

       [[48, 13]],

       [[49, 14]],

       [[50, 14]],

       [[52, 16]],

       [[52, 17]],

       [[51, 18]],

       [[49, 16]],

       [[48, 16]],

       [[47, 15]]], dtype=int32), array([[[45, 13]],

       [[46, 12]],

       [[47, 12]],

       [[48, 13]],

       [[47, 14]],

       [[46, 14]]], dtype=int32), array([[[38,  0]],

       [[37,  1]],

       [[30,  1]],

       [[29,  2]],

       [[26,  2]],

       [[25,  3]],

       [[22,  3]],

       [[21,  4]],

       [[19,  4]],

       [[19,  6]],

       [[17,  8]],

       [[16,  8]],

       [[15,  9]],

       [[13,  9]],

       [[11,  7]],

       [[10,  8]],

       [[ 9,  7]],

       [[ 8,  7]],

       [[ 7,  8]],

       [[ 4,  8]],

       [[ 3,  7]],

       [[ 1,  7]],

       [[ 0,  8]]

[array([[[11, 23]],

       [[12, 22]],

       [[13, 22]],

       [[14, 23]],

       [[12, 25]],

       [[12, 26]],

       [[11, 27]],

       [[ 7, 27]],

       [[ 6, 26]],

       [[ 9, 23]]], dtype=int32), array([[[21, 20]],

       [[22, 19]],

       [[27, 19]],

       [[28, 20]],

       [[32, 20]],

       [[33, 21]],

       [[36, 21]],

       [[37, 22]],

       [[38, 22]],

       [[39, 23]],

       [[40, 23]],

       [[41, 24]],

       [[42, 24]],

       [[43, 25]],

       [[44, 25]],

       [[45, 26]],

       [[46, 26]],

       [[50, 30]],

       [[51, 30]],

       [[53, 32]],

       [[51, 34]],

       [[49, 34]],

       [[48, 33]],

       [[47, 34]],

       [[45, 34]],

       [[44, 33]],

       [[43, 33]],

       [[42, 32]],

       [[41, 32]],

       [[40, 31]],

       [[39, 31]],

       [[38, 30]],

       [[38, 29]],

       [[35, 29]],

       [[34, 30]],

       [[33, 30]],

       [[33, 31]],

       [[32, 32]],

       [[31, 32]],

     

[array([[[ 0,  0]],

       [[ 0, 11]],

       [[ 1, 10]],

       [[ 1,  9]],

       [[ 2,  8]],

       [[ 3,  8]],

       [[ 6,  5]],

       [[ 9,  5]],

       [[10,  6]],

       [[11,  6]],

       [[12,  7]],

       [[14,  7]],

       [[16,  9]],

       [[16, 10]],

       [[14, 12]],

       [[10, 12]],

       [[ 9, 11]],

       [[ 7, 11]],

       [[ 6, 12]],

       [[ 5, 12]],

       [[ 4, 13]],

       [[ 2, 13]],

       [[ 2, 14]],

       [[ 1, 15]],

       [[ 0, 15]],

       [[ 0, 21]],

       [[21, 21]],

       [[21,  0]]], dtype=int32)]
[array([[[12, 22]],

       [[13, 21]],

       [[15, 21]],

       [[16, 22]],

       [[13, 25]],

       [[10, 25]],

       [[ 9, 24]],

       [[11, 22]]], dtype=int32), array([[[18, 20]],

       [[19, 19]],

       [[32, 19]],

       [[33, 20]],

       [[35, 20]],

       [[36, 21]],

       [[38, 21]],

       [[39, 22]],

       [[41, 22]],

       [[43, 24]],

       [[44, 24]],

       [[45, 25]],

       [[4

In [10]:
import math 
import cv2 
import numpy as np

eye_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml') 
if eye_cascade.empty():
    raise IOError('Unable to load the eye cascade classifier xml file')

cap = cv2.VideoCapture('facevideo.mp4')
ds_factor = 0.5
ret, frame = cap.read()
contours = []

while True: 
    ret, frame = cap.read() 
    frame = cv2.resize(frame, None, fx=ds_factor, fy=ds_factor, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=1)
    for (x_eye, y_eye, w_eye, h_eye) in eyes:
        pupil_frame = gray[y_eye:y_eye + h_eye, x_eye:x_eye + w_eye]
        ret, thresh = cv2.threshold(pupil_frame, 80, 255, cv2.THRESH_BINARY)
        cv2.imshow("threshold", thresh)
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        print(contours)

    for contour in contours:
        area = cv2.contourArea(contour)
        rect = cv2.boundingRect(contour)
        x, y, w, h = rect
        radius = 0.15 * (w + h)

        area_condition = (100 <= area <= 200)
        symmetry_condition = (abs(1 - float(w) / float(h)) <= 0.2)
        fill_condition = (abs(1 - (area / (math.pi * math.pow(radius, 2.0)))) <= 0.4)
        cv2.circle(frame, (int(x_eye + x + radius), int(y_eye + y + radius)), int(1.3 * radius), (0, 180, 0), -1)

    cv2.imshow('Pupil Detector', frame)
    c = cv2.waitKey(1) 
    if c == 27: 
        break
  
cap.release() 
cv2.destroyAllWindows()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[array([[[92,  7]],

       [[92,  9]],

       [[91, 10]],

       [[84, 10]],

       [[84, 17]],

       [[88, 17]],

       [[89, 18]],

       [[92, 18]],

       [[94, 20]],

       [[96, 20]],

       [[96,  7]]], dtype=int32)]
[]
[]
[]
[]
[]
[array([[[47, 52]],

       [[47, 54]],

       [[54, 54]],

       [[54, 52]]], dtype=int32), array([[[36, 52]],

       [[34, 54]],

       [[31, 54]],

       [[38, 54]],

       [[38, 52]]], dtype=int32)]
[]
[]
[array([[[ 7, 48]],

       [[ 7, 51]],

       [[10, 51]],

       [[10, 48]]], dtype=int32), array([[[51, 45]],

       [[49, 47]],

       [[47, 47]],

       [[46, 48]],

       [[43, 48]],

       [[42, 49]],

       [[35, 49]],

       [[34, 50]],

       [[31, 50]],

       [[30, 51]],

       [[23, 51]],

       [[21, 49]],

       [[17, 49]],

       [[17, 50]],

       [[16, 51]],

       [[16, 52]],

       [[52, 52]],

       [[52, 45]]], dtype=int32), array([[[28, 12]],

    

[array([[[ 0, 34]],

       [[ 0, 54]],

       [[54, 54]],

       [[54, 40]],

       [[53, 41]],

       [[52, 41]],

       [[48, 45]],

       [[43, 45]],

       [[42, 46]],

       [[39, 46]],

       [[38, 47]],

       [[30, 47]],

       [[29, 48]],

       [[22, 48]],

       [[21, 47]],

       [[21, 48]],

       [[20, 49]],

       [[19, 48]],

       [[17, 48]],

       [[16, 47]],

       [[15, 47]],

       [[14, 46]],

       [[13, 46]],

       [[12, 45]],

       [[11, 45]],

       [[10, 44]],

       [[ 9, 44]],

       [[ 2, 37]],

       [[ 2, 36]]], dtype=int32), array([[[29, 11]],

       [[28, 12]],

       [[27, 12]],

       [[24, 15]],

       [[25, 15]],

       [[29, 19]],

       [[30, 19]],

       [[31, 20]],

       [[33, 20]],

       [[34, 21]],

       [[40, 21]],

       [[42, 23]],

       [[43, 23]],

       [[44, 24]],

       [[45, 24]],

       [[46, 25]],

       [[48, 25]],

       [[47, 24]],

       [[47, 23]],

       [[42, 18]],

     

[array([[[ 0, 33]],

       [[ 0, 49]],

       [[49, 49]],

       [[49, 35]],

       [[46, 35]],

       [[45, 36]],

       [[43, 36]],

       [[42, 37]],

       [[40, 37]],

       [[39, 38]],

       [[35, 38]],

       [[34, 39]],

       [[32, 39]],

       [[31, 40]],

       [[27, 40]],

       [[26, 41]],

       [[15, 41]],

       [[14, 40]],

       [[ 6, 40]],

       [[ 5, 39]],

       [[ 5, 38]],

       [[ 4, 37]],

       [[ 4, 36]],

       [[ 2, 34]],

       [[ 1, 34]]], dtype=int32), array([[[25,  3]],

       [[23,  5]],

       [[22,  5]],

       [[21,  6]],

       [[19,  6]],

       [[18,  7]],

       [[17,  7]],

       [[17,  9]],

       [[16, 10]],

       [[16, 11]],

       [[15, 12]],

       [[15, 14]],

       [[16, 15]],

       [[27, 15]],

       [[28, 14]],

       [[30, 14]],

       [[31, 13]],

       [[35, 13]],

       [[36, 14]],

       [[40, 14]],

       [[40, 10]],

       [[38,  8]],

       [[38,  7]],

       [[37,  6]],

     

[array([[[50, 31]],

       [[49, 32]],

       [[46, 32]],

       [[45, 33]],

       [[38, 33]],

       [[37, 32]],

       [[36, 33]],

       [[35, 33]],

       [[34, 34]],

       [[31, 34]],

       [[31, 35]],

       [[32, 34]],

       [[35, 34]],

       [[36, 35]],

       [[38, 35]],

       [[39, 36]],

       [[41, 36]],

       [[42, 37]],

       [[40, 39]],

       [[39, 39]],

       [[38, 40]],

       [[36, 40]],

       [[35, 41]],

       [[34, 41]],

       [[33, 42]],

       [[13, 42]],

       [[12, 41]],

       [[11, 41]],

       [[10, 40]],

       [[ 8, 40]],

       [[ 7, 39]],

       [[ 6, 39]],

       [[ 4, 37]],

       [[ 2, 37]],

       [[ 0, 35]],

       [[ 0, 50]],

       [[50, 50]]], dtype=int32), array([[[ 0, 29]],

       [[ 0, 33]],

       [[ 0, 32]],

       [[ 1, 31]],

       [[ 1, 30]]], dtype=int32), array([[[26,  9]],

       [[27,  8]],

       [[36,  8]],

       [[37,  9]],

       [[40,  9]],

       [[41, 10]],

       [[43

[array([[[  0,  45]],

       [[  0, 124]],

       [[ 26, 124]],

       [[ 26, 120]],

       [[ 27, 119]],

       [[ 27, 112]],

       [[ 26, 112]],

       [[ 25, 111]],

       [[ 25, 109]],

       [[ 24, 108]],

       [[ 24, 107]],

       [[ 21, 104]],

       [[ 21, 103]],

       [[ 20, 102]],

       [[ 18, 102]],

       [[ 17, 101]],

       [[ 16, 101]],

       [[ 15, 100]],

       [[ 15,  99]],

       [[ 12,  96]],

       [[ 12,  95]],

       [[ 11,  94]],

       [[ 11,  92]],

       [[ 10,  91]],

       [[ 10,  90]],

       [[  9,  89]],

       [[  9,  88]],

       [[  8,  87]],

       [[  8,  86]],

       [[  7,  85]],

       [[  7,  79]],

       [[  6,  78]],

       [[  6,  64]],

       [[  7,  63]],

       [[  7,  60]],

       [[  8,  59]],

       [[  6,  57]],

       [[  5,  57]],

       [[  3,  55]],

       [[  3,  51]],

       [[  2,  50]],

       [[  2,  49]],

       [[  1,  48]],

       [[  1,  47]],

       [[  0,  46]]], dtype=int

[array([[[ 2,  7]],

       [[ 1,  8]],

       [[ 0,  8]],

       [[ 0, 18]],

       [[ 2, 18]],

       [[ 3, 17]],

       [[13, 17]],

       [[14, 16]],

       [[17, 16]],

       [[18, 17]],

       [[20, 17]],

       [[20, 15]],

       [[15, 10]],

       [[13, 10]],

       [[11,  8]],

       [[ 8,  8]],

       [[ 7,  7]]], dtype=int32), array([[[40,  0]],

       [[40, 21]],

       [[38, 23]],

       [[38, 24]],

       [[37, 25]],

       [[36, 25]],

       [[30, 19]],

       [[29, 19]],

       [[29, 25]],

       [[26, 28]],

       [[25, 27]],

       [[23, 27]],

       [[22, 26]],

       [[21, 26]],

       [[19, 24]],

       [[17, 24]],

       [[12, 29]],

       [[11, 29]],

       [[10, 30]],

       [[ 8, 30]],

       [[ 5, 33]],

       [[ 5, 35]],

       [[ 4, 36]],

       [[ 9, 36]],

       [[10, 37]],

       [[11, 37]],

       [[12, 38]],

       [[11, 39]],

       [[ 9, 39]],

       [[ 8, 40]],

       [[ 5, 40]],

       [[ 5, 41]],

     

[array([[[41, 75]],

       [[43, 77]],

       [[43, 78]],

       [[42, 79]],

       [[41, 79]],

       [[39, 77]]], dtype=int32), array([[[34, 43]],

       [[35, 42]],

       [[39, 42]],

       [[40, 43]],

       [[39, 44]],

       [[39, 45]],

       [[37, 47]],

       [[36, 47]],

       [[32, 51]],

       [[31, 51]],

       [[27, 55]],

       [[26, 55]],

       [[25, 56]],

       [[25, 59]],

       [[27, 61]],

       [[29, 61]],

       [[30, 60]],

       [[32, 60]],

       [[34, 58]],

       [[36, 58]],

       [[37, 57]],

       [[38, 57]],

       [[39, 56]],

       [[41, 56]],

       [[42, 55]],

       [[51, 55]],

       [[52, 54]],

       [[57, 54]],

       [[58, 55]],

       [[62, 55]],

       [[63, 56]],

       [[67, 56]],

       [[68, 57]],

       [[68, 58]],

       [[69, 58]],

       [[71, 60]],

       [[71, 62]],

       [[70, 63]],

       [[66, 63]],

       [[63, 60]],

       [[62, 60]],

       [[61, 59]],

       [[59, 59]],

     

In [25]:
import math 
import cv2 
import numpy as np

eye_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml')

if eye_cascade.empty():
    raise IOError('Unable to load the eye cascade classifier xml file')

cap = cv2.VideoCapture('facevideo.mp4')

ds_factor = 0.5
ret, frame = cap.read()
corners = cv2.goodFeaturesToTrack(cap, 7, 0.05, 25) 
corners = np.float32(corners)
contours = []

while True: 
    ret, frame = cap.read() 
    frame = cv2.resize(frame, None, fx=ds_factor, fy=ds_factor, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=1)
    
    for (x_eye, y_eye, w_eye, h_eye) in eyes:
        pupil_frame = gray[y_eye:y_eye + h_eye, x_eye:x_eye + w_eye]
        ret, thresh = cv2.threshold(pupil_frame, 80, 255, cv2.THRESH_BINARY)
        cv2.imshow("threshold", thresh)
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        print(contours)

    for item in corners:
        x, y = item[0]
        cv2.circle(img, (x,y), 5, 255, -1)
        
    cv2.imshow('Pupil Detector', frame)
    c = cv2.waitKey(1) 
    if c == 27: 
        break
  
cap.release() 
cv2.destroyAllWindows()

TypeError: Required argument 'mask' (pos 5) not found

In [3]:
#이미지에서 표정
import cv2

facePath = "D:javaForever/util/opencv/sources/data/lbpcascades/lbpcascade_frontalface.xml"
faceCascade = cv2.CascadeClassifier(facePath)

smilePath = "D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_smile.xml"
smileCascade = cv2.CascadeClassifier(smilePath)

img = cv2.imread("face.jpg")  
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor= 1.1,
    minNeighbors=8,
    minSize=(55, 55),
    flags=cv2.CASCADE_SCALE_IMAGE
)

for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    smile = smileCascade.detectMultiScale(
        roi_gray,
        scaleFactor= 1.16,
        minNeighbors=35,
        minSize=(25, 25),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    for (x2, y2, w2, h2) in smile:
        cv2.rectangle(roi_color, (x2, y2), (x2+w2, y2+h2), (255, 0, 0), 2)
        cv2.putText(img,'Smile',(x,y-7), 3, 1.2, (0, 255, 0), 2, cv2.LINE_AA)

cv2.imshow('Smile', img)
#cv2.imwrite("smile.jpg",img)
c = cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#영상에서 표정

import cv2
import sys
import numpy as np

faceCascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_alt.xml')
eyeCascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml')
mouthCascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_mcs_mouth.xml')

video_capture = cv2.VideoCapture('facevideo.mp4')

def findMouth(frame, eyes, faces):
    mouths = mouthCascade.detectMultiScale(
            frame,
            scaleFactor=1.2,
            minNeighbors=8,
            minSize=(30, 10),
            flags = cv2.CASCADE_SCALE_IMAGE)

    if len(mouths) <= 0:
        return 0, 0, 0, 0

    eyeBottom = 0

    for eye in eyes:
        eyeX, eyeY, eyeW, eyeH = eye
        if (eyeY + eyeH) > eyeBottom:
            eyeBottom = eyeY + eyeH

    faceX, faceY, faceW, faceH = faces[0]
    faceBottom = faceY + faceH

    for mouth in mouths:
        mouthX, mouthY, mouthW, mouthH = mouth
        if mouthY > (eyeBottom + 50) and (mouthY + mouthH) < faceBottom:
            return mouth

    return 0, 0, 0, 0

if __name__ == '__main__':
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.equalizeHist(gray)

        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=6,
            minSize=(50, 50),
            flags = cv2.CASCADE_SCALE_IMAGE)

        eyes = eyeCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=6,
            minSize=(50, 50),
            flags = cv2.CASCADE_SCALE_IMAGE)


        # Draw a rectangle around the faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
        # Draw a rectangle around the eyes
        if len(eyes) >= 2:
            eyes = eyes[:2]
        for (x, y, w, h) in eyes:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
        # Draw a rectangle around the mouth
        if len(faces) > 0 and len(eyes) > 0:
            x, y, w, h = findMouth(gray2, eyes, faces)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(video_capture,'Smile',(x,y-7), 3, 1.2, (0, 255, 0), 2, cv2.LINE_AA)
        # Display the resulting frame
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()